In [ ]:
import gdown
%pip install accelerate -U

try:
  dataset_url = "https://drive.google.com/file/d/1jy5uUVQRwXOLrjM7LvzK0NCgDOpDigHQ/view?usp=sharing"
  zip_download_dir =  "artifacts/"
  file_id = dataset_url.split("/")[-2]
  prefix = "https://drive.google.com/uc?/export=download&id="
  gdown.download(prefix + file_id, zip_download_dir)

except Exception as e:
  raise e

Downloading...
From: https://drive.google.com/uc?/export=download&id=1jy5uUVQRwXOLrjM7LvzK0NCgDOpDigHQ
To: /content/artifacts/sunbird-ug-lang-v1.0.jsonl.txt
100%|██████████| 10.4M/10.4M [00:00<00:00, 33.5MB/s]


In [ ]:
import pandas as pd
import json

data_file = "artifacts/sunbird-ug-lang-v1.0.jsonl.txt"
with open(data_file, "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file.readlines()]

english_texts = []
luganda_texts = []

def preprocess_text(text):
    return text.lower()

for record in data:
    luganda_text = preprocess_text(record.get('Luganda'))
    english_text = preprocess_text(record.get('English'))
    luganda_texts.append(luganda_text)
    english_texts.append(english_text)

df = pd.DataFrame({
    'English': english_texts,
    'Luganda': luganda_texts
})


In [ ]:
df

,English,Luganda
0,eggplants always grow best under warm conditions.,bbiringanya lubeerera asinga kukulira mu mbee...
1,farmland is sometimes a challenge to farmers.,ettaka ly'okulimirako n'okulundirako ebiseera ...
2,farmers should be encouraged to grow more coffee.,abalimi balina okukubirizibwa okwongera okulim...
3,uganda is focusing on farming.,uganda essira eritadde ku bulimi.
4,some plants die due to lack of sunlight.,ebimera ebimu bifa olw'ebbula ly'omusana.
...,...,...
25001,there is usually heavy traffic in the morning ...,ebidduka bitera okuba ebingi ku makya n'akawun...
25002,he promised to clear the debt by the end of th...,yasuubiza okumalayo ebbanja ku nkomerero y'omw...
25003,a traffic police officer was knocked down by a...,owa poliisi y'ebidduka yatomeddwa emmotoka eya...
25004,he lost his sight when he was still a baby.,yaziba amaaso bwe yali akyali muwere.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-lg")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-lg")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
from sklearn.model_selection import train_test_split

train_size = int(0.8 * len(df))
val_size = int(0.1 * len(df))
test_size = len(df) - train_size - val_size

train_df = df.iloc[:train_size]
val_df = df.iloc[train_size:train_size + val_size]
test_df = df.iloc[train_size + val_size:]